## Designing and Creating a Database

In this project, we will be working with a file of Major League Baseball games from [Retrosheet](https://www.retrosheet.org/). 

## Getting to Know the Data

Retrosheet compiles detailed statistics on baseball games from the 1800s through to today. The main file we will be working from game_log.csv, has been produced by combining 127 separate CSV files from retrosheet, and has been pre-cleaned to remove some inconsistencies. The game log has hundreds of data points on each game which we will normalize into several separate tables using SQL, providing a robust database of game-level statistics.

In addition to the main file, we have also included three 'helper' files, also sourced from Retrosheet:

   + park_codes.csv
   + person_codes.csv
   + team_codes.csv

These three helper files in some cases contain extra data, but will also make things easier as they will form the basis for three of our normalized tables.

An important first step when working with any new data is to perform exploratory data analysis (EDA). EDA gets us familiar with the data and gives us a level of background knowledge that will help us throughout our project. The methods we use when performing EDA will depend on what we plan to do with the data. In this case, we're wanting to create a normalized database, so our focus should be:

   + Becoming familiar, at a high level, with the meaning of each column in each file.
   + Thinking about the relationships between columns within each file.
   + Thinking about the relationships between columns across different files.

A game_log_fields.txt file is included from Retrosheet which explains the fields included in our main file, which will be useful to assist our EDA.

Let's get started exploring the data by using pandas to read and explore the data.

In [1]:
import pandas as pd
pd.set_option('max_columns', 180)
pd.set_option('max_rows', 200000)
pd.set_option('max_colwidth', 5000)
games = pd.read_csv("game_log.csv", low_memory = False)
print(games.shape)
print(games.head())

(171907, 161)
       date  number_of_game day_of_week v_name v_league  v_game_number h_name  \
0  18710504               0         Thu    CL1      NaN              1    FW1   
1  18710505               0         Fri    BS1      NaN              1    WS3   
2  18710506               0         Sat    CL1      NaN              2    RC1   
3  18710508               0         Mon    CL1      NaN              3    CH1   
4  18710509               0         Tue    BS1      NaN              2    TRO   

  h_league  h_game_number  v_score  h_score  length_outs day_night completion  \
0      NaN              1        0        2         54.0         D        NaN   
1      NaN              1       20       18         54.0         D        NaN   
2      NaN              1       12        4         54.0         D        NaN   
3      NaN              1       12       14         54.0         D        NaN   
4      NaN              1        9        5         54.0         D        NaN   

  forefeit p

In [2]:
#Annotation of the column names
!cat game_log_fields.txt

Field(s)  Meaning
    1     Date in the form "yyyymmdd"
    2     Number of game:
             "0" -- a single game
             "1" -- the first game of a double (or triple) header
                    including seperate admission doubleheaders
             "2" -- the second game of a double (or triple) header
                    including seperate admission doubleheaders
             "3" -- the third game of a triple-header
             "A" -- the first game of a double-header involving 3 teams
             "B" -- the second game of a double-header involving 3 teams
    3     Day of week  ("Sun","Mon","Tue","Wed","Thu","Fri","Sat")
  4-5     Visiting team and league
    6     Visiting team game number
          For this and the home team game number, ties are counted as
          games and suspended games are counted from the starting
          rather than the ending date.
  7-8     Home team and league
    9     Home team game number
10-11     Visiting and home team score (unquoted)


It seems that there are more than 170000 games recorded in main game file. This file includes the information about the games, scores and also related teams and players information.

In [3]:
parks = pd.read_csv("park_codes.csv")
print(parks.shape)
print(parks.head())

(252, 9)
  park_id                           name  \
0   ALB01                 Riverside Park   
1   ALT01                  Columbia Park   
2   ANA01       Angel Stadium of Anaheim   
3   ARL01              Arlington Stadium   
4   ARL02  Rangers Ballpark in Arlington   

                                        aka       city state       start  \
0                                       NaN     Albany    NY  09/11/1880   
1                                       NaN    Altoona    PA  04/30/1884   
2             Edison Field; Anaheim Stadium    Anaheim    CA  04/19/1966   
3                                       NaN  Arlington    TX  04/21/1972   
4  The Ballpark in Arlington; Ameriquest Fl  Arlington    TX  04/11/1994   

          end league                                           notes  
0  05/30/1882     NL  TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882  
1  05/31/1884     UA                                             NaN  
2         NaN     AL                                    

It looks like that park_ids in parks file are linked with park_ids in the main game file. Other informations, like states, leagues and start and end time are also included in this file. 

In [4]:
persons = pd.read_csv("person_codes.csv")
print(persons.shape)
print(persons.head())

(20494, 7)
         id     last   first player_debut mgr_debut coach_debut ump_debut
0  aardd001  Aardsma   David   04/06/2004       NaN         NaN       NaN
1  aaroh101    Aaron    Hank   04/13/1954       NaN         NaN       NaN
2  aarot101    Aaron  Tommie   04/10/1962       NaN  04/06/1979       NaN
3  aased001     Aase     Don   07/26/1977       NaN         NaN       NaN
4  abada001     Abad    Andy   09/10/2001       NaN         NaN       NaN


Persons file contains information about different roles. Their names match the names in main game file. There are manager and coaches in a baseball team. After researching with Google and Wikipedia, manager is head coach, who decides game strategies and player substitutions. While coaches are assistants to the manager, who help the manager to run the team. However, there is no team informations related to these persons.

In [5]:
teams = pd.read_csv("team_codes.csv")
print(teams.shape)
print(teams.head())

(150, 8)
  team_id league  start   end       city         nickname franch_id  seq
0     ALT     UA   1884  1884    Altoona  Mountain Cities       ALT    1
1     ARI     NL   1998     0    Arizona     Diamondbacks       ARI    1
2     BFN     NL   1879  1885    Buffalo           Bisons       BFN    1
3     BFP     PL   1890  1890    Buffalo           Bisons       BFP    1
4     BL1    NaN   1872  1874  Baltimore         Canaries       BL1    1


Teams file have team_ids, which are linked with league name in main game file. 

In [6]:
teams['league'].value_counts(dropna = False)

NL     45
NaN    26
AL     25
AA     24
UA     13
FL      9
PL      8
Name: league, dtype: int64

There are six leagues in total. About 20% of the teams don't have offical link to any leagues. These independent teams are members of Minor League.

There is one column called 'franch_id'. It seems to be related with team_id. Let's explore their relationship.

In [7]:
teams['franch_id'].value_counts()

BS1    4
BR3    3
SE1    3
LAA    3
PHA    3
TRN    3
MLA    3
BL2    2
PH1    2
PT1    2
SL2    2
HR1    2
CH2    2
WS1    2
MON    2
SLU    2
BSP    2
CN2    2
CL3    2
FLO    2
NY2    2
BLA    2
SL4    2
LS2    2
WS2    2
WS9    2
IND    2
CL2    1
RIC    1
WSU    1
BUF    1
PHP    1
CNU    1
KCU    1
DTN    1
WS7    1
DET    1
BOS    1
PHU    1
WS3    1
KC2    1
TL2    1
NYN    1
SLF    1
WS6    1
TL1    1
TBA    1
PRO    1
WIL    1
NY4    1
CLE    1
BR1    1
TOR    1
IN2    1
CN1    1
BRP    1
MLU    1
PTF    1
PH3    1
RC1    1
SDN    1
ML3    1
BRF    1
WS5    1
ML1    1
BSU    1
ALT    1
WOR    1
PTP    1
BFN    1
CHU    1
BR4    1
ELI    1
BR2    1
BFP    1
ARI    1
LS1    1
WS4    1
NYP    1
WS8    1
KCA    1
FW1    1
SPU    1
PHI    1
KEO    1
SR2    1
CN3    1
SEA    1
BL1    1
KCF    1
BLF    1
CHA    1
BLU    1
CHP    1
SL1    1
PTU    1
CL1    1
PH4    1
RC2    1
MID    1
COL    1
TRO    1
CL6    1
SR1    1
NH1    1
PH2    1
CL5    1
KCN    1
HOU    1
CLP    1
IN3    1
B

In [8]:
print(teams[teams['franch_id']=='BS1'])

   team_id league  start   end       city nickname franch_id  seq
21     BS1    NaN   1871  1875     Boston   Braves       BS1    1
22     BSN     NL   1876  1952     Boston   Braves       BS1    2
23     MLN     NL   1953  1965  Milwaukee   Braves       BS1    3
24     ATL     NL   1966     0    Atlanta   Braves       BS1    4


This reveals that the operations of teams change along the time. Some of the teams with the id of BS1 belonged to Minor league at the beginning. Later they transferred to one of the Major Leagues. When this happened, their team_id changed. Franch_id provides information about this kind of change.

#### Defensive position numbers

After researching with Google, we know that baseball has different defensive positions. And each defensive position has a unique number. Here are a list of all the defensive positions.
   + 1. Pitcher
   + 2. Catcher
   + 3. First Baseman
   + 4. Second Baseman
   + 5. Third Baseman
   + 6. Shortstop
   + 7. Left Fielder
   + 8. Center Fielder
   + 9. Right Fielder

A defensive position number chart of the baseball field is shown here.
![](https://baseballmadefun.com/wp-content/uploads/2017/03/positionnumberchart-e1489511629471.jpg)

#### Leagues

Let's explore the leagues in the main game file.

In [9]:
games['h_league'].value_counts(dropna = False)

NL     88867
AL     74712
AA      5039
FL      1243
NaN     1086
PL       532
UA       428
Name: h_league, dtype: int64

Same with what we found in teams file, there are six leagues in total. The majority of the games happened in NL and AL. Some of the teams don't belong to any leagues, whose league value is NaN. Google and Wikipedia show the leagues they represent as follows:
   + NL: National League
   + AL: American League
   + AA: American Association
   + FL: Federal League
   + PL: Players League
   + UA: Union Association

## Importing Data into SQLite

We have got farmilar with all the files. To create a normalized database from the four files, we need to import all the data into SQLite. 

To insert data into a noramalized database, we'll need a single column that can be used as a primary key. The game log file does not have a single column that can be used as a primary key to uniquely identify each game. There are three ways that we could handle this:

   + Make a compound primary key, such as a primary key of the date, h_name, and number_of_game columns.
   + Insert an integer primary key, eg where the first row is 1, the second row is 2, etc.
   + Insert a new column using a custom format.

Because we have not yet normalized our data, it's better not to start with a compound primary key - if we do this, we might end up needing to create a compound key in another table that includes this compound key, which would quickly become cumbersome to work with. An integer primary key is a good choice, but we should first explore whether Retrosheet already have a system for uniquely identifying each game. If they do, this is a better option. It means that if at some later stage we choose to incorporate more detailed game data into our database, the keys we use will be compatible with other sources.

Exploring the Retrosheet site, we can find a [data dictionary](https://www.retrosheet.org/eventfile.htm) for their event files, which list every event within each game. This includes the following description:

id: Each game begins with a twelve character ID record which identifies the date, home team, and number of the game. For example, ATL198304080 should be read as follows. The first three characters identify the home team (the Braves). The next four are the year (1983). The next two are the month (April) using the standard numeric notation, 04, followed by the day (08). The last digit indicates if this is a single game (0), first game (1) or second game (2) if more than one game is played during a day, usually a double header The id record starts the description of a game thus ending the description of the preceding game in the file.

This essentially makes a custom key using the three columns we identified in our composite key example earlier. After we import the data, we'll construct this column to use as a primary key in our final database.

Our next task is to import the data into SQLite. Because we've already read the data into dataframes, we'll use pandas library to import the data.

In [10]:
import sqlite3
import numpy as np

#Create a function that takes a SQL query and returns pandas dataframes of that query
def run_query(q):
    with sqlite3.connect('mlb.db') as conn:
        return pd.read_sql(q, conn)

#Create a function that takes a SQL command and executes it using the sqlite module
def run_command(c):
    with sqlite3.connect('mlb.db') as conn:
        #Tell SQLite to autocommit any changes
        conn.isolation_level = None
        conn.execute(c)
conn = sqlite3.connect('mlb.db')
tables = {"game_log":games,
          "park_codes":parks,
          "person_codes":persons,
          "team_codes":teams}
for key, value in tables.items():
    conn.execute("DROP TABLE IF EXISTS {};".format(key))
    value.to_sql(key, conn, index = False)
    
def show_tables():
    q = '''
    SELECT
        name,
        type
    FROM sqlite_master
    WHERE type IN ('table', 'view');
    '''
    return run_query(q)

In [11]:
show_tables()

,name,type
0,league,table
1,appearance_type,table
2,team,table
3,game,table
4,game_team,table
5,park,table
6,person,table
7,person_appearance,table
8,game_log,table
9,person_codes,table


Now let's create a new column called game_id in the game_log table to specify each game. Then use SQL string concatenation to update the new columns with a unique ID using the Retrosheet format outlined above.

In [12]:
c = '''
ALTER TABLE game_log
ADD COLUMN game_id TEXT;
'''
game_log = run_command(c)

In [13]:
c_upd1 = '''
UPDATE game_log
SET game_id = h_name || CAST(date as TEXT) || CAST(number_of_game as TEXT);
'''
game_log = run_command(c_upd1)

In [14]:
q = '''
    SELECT 
        game_id,
        date,
        h_name
    FROM game_log
    LIMIT 5;
    '''
run_query(q)

,game_id,date,h_name
0,FW1187105040,18710504,FW1
1,WS3187105050,18710505,WS3
2,RC1187105060,18710506,RC1
3,CH1187105080,18710508,CH1
4,TRO187105090,18710509,TRO


## Looking for Normalization Opportunities

In this step, we're going to look for specific opportunities to normalize our data by reducing repetition. Here are two examples of repetition we can find and remove:

#### Repetition in columns

Let's look at the following segment of data:

| v_player_1_id | v_player_1_name   | v_player_1_def_pos | v_player_2_id | v_player_2_name  | v_player_2_def_pos |
|---------------|-------------------|--------------------|---------------|------------------|--------------------|
| villj001      | Jonathan Villar   | 5.0                | genns001      | Scooter Gennett  | 4.0                |
| granc001      | Curtis Granderson | 8.0                | cabra002      | Asdrubal Cabrera | 6.0                |
| kendh001      | Howie Kendrick    | 7.0                | turnj001      | Justin Turner    | 5.0                |
| jasoj001      | John Jaso         | 3.0                | polag001      | Gregory Polanco  | 9.0                |
| gordd002      | Dee Gordon        | 4.0                | telit001      | Tomas Telis      | 2.0                |

We have three columns that relate to one player, followed by three columns that relate to another player. We could restructure our data to remove this repetition - we would need to add an extra column to include the data that was previously only contained in the name of the column:

| id       | name              | def_pos | off_pos |
|----------|-------------------|---------|---------|
| villj001 | Jonathan Villar   | 5.0     | 1.0     |
| granc001 | Curtis Granderson | 8.0     | 1.0     |
| kendh001 | Howie Kendrick    | 7.0     | 1.0     |
| jasoj001 | John Jaso         | 3.0     | 1.0     |
| gordd002 | Dee Gordon        | 4.0     | 1.0     |
| genns001 | Scooter Gennett   | 4.0     | 2.0     |
| cabra002 | Asdrubal Cabrera  | 6.0     | 2.0     |
| turnj001 | Justin Turner     | 5.0     | 2.0     |
| polag001 | Gregory Polanco   | 9.0     | 2.0     |
| telit001 | Tomas Telis       | 2.0     | 2.0     |

#### Non-primary key columns should be attributes of the primary key

The primary key of our game log is our game_id, but the players name are not attributes of a game, but of the player id. If the only data we had was the game log, we would remove this column and create a new table that had the names of each player. As it happens, our person_codes table already has a list of our player IDs and names, so we can remove these without the need for creating a new table first.

#### Redundant Data

Lastly, we want to eliminate any redundant data - that is, columns where the data is available elsewhere. A good example of this can be found in our park_codes table, which will form the basis of our eventual park table. Let's look at the first few rows (we won't display the notes column as is not relevant to our discussion):

| park_id |              name             |                    aka                   |    city   | state |    start   |     end    | league |
|:-------:|:-----------------------------:|:----------------------------------------:|:---------:|:-----:|:----------:|:----------:|:------:|
|  ALB01  |         Riverside Park        |                    NaN                   |   Albany  |   NY  | 09/11/1880 | 05/30/1882 |   NL   |
|  ALT01  |         Columbia Park         |                    NaN                   |  Altoona  |   PA  | 04/30/1884 | 05/31/1884 |   UA   |
|  ANA01  |    Angel Stadium of Anaheim   |       Edison Field; Anaheim Stadium      |  Anaheim  |   CA  | 04/19/1966 |     NaN    |   AL   |
|  ARL01  |       Arlington Stadium       |                    NaN                   | Arlington |   TX  | 04/21/1972 | 10/03/1993 |   AL   |
|  ARL02  | Rangers Ballpark in Arlington | The Ballpark in Arlington; Ameriquest Fl | Arlington |   TX  | 04/11/1994 |     NaN    |   AL   |

The start and end columns show the first and last games played at the park, however we will be able to derive this information by looking at the park information for each game in game_log table. Similarly, the league information is going to be available elsewhere in our database.

The followings are opportunities for normalization of our data:
   
   + In park_codes, the start date, end date and league information can be reproduced from game_log table;
   + In person_codes, all the debut data can be reproduced from game_log table;
   + In team_codes, the start, end and sequence columns can be able to be reproduced from game_log table;
   + In game_log, id and name of players occur at the same time at lots of places. We can only keep the id information, and get access players' name from person_codes table;
   + In game_log, all offensive and defensive stats are repeated for the home team and visiting team. We should break these information and create a table to record each game twice, one for each team;
   + Similarly, in game_log, we have 9 players for each team with their position numbers. We can remove these information from game_log, and create a new table to list the players' appearence and position numbers; 
   + We can do the similar thing for umpire information in game_log;
   + We can seperate the award information from game_log to a new table or to person_codes.

## Planning a Normalized Schema

Now that we've started to think about normalization ideas, it's time to start planning our schema. The best way to work is visually with a schema diagram, just like the ones we've used so far in this course. Start by creating a diagram of the four existing tables and their columns, and then gradually create new tables that move the data into a more normalized state.

In this project we will use a schema designing tool like [DbDesigner.net](https://www.dbdesigner.net/). This free tool allows us to create a schema and will create lines to show foreign key relations clearly.

Here is the schema we created with the above tool.
[![dataquest-baseball-game-schema.png](https://i.postimg.cc/KYTry4QJ/dataquest-baseball-game-schema.png)](https://postimg.cc/cKs8Rxpn)

## Creating Tables Without Foreign Key Relations

As we work through creating the tables in this schema, we'll talk about why we made particular choices during the normalization process. We'll start by creating the tables that don't contain any foreign key relations. It's important to start with these tables, as other tables will have relations to these tables, and so these tables will need to exist first.

The tables we will create are below, with some notes on the normalization choices made:

   + person: Each of the 'debut' columns have been omitted, as the data will be able to be found from other tables.
   + park: The start, end, and league columns contain data that is found in the main game log and can be removed.
   + team: The start, end and sequence columns include data that have matches in the main game log and can be removed.
   
We'll first create each table, and then we'll insert the data.

In [15]:
#Create the park table with columns and primary key as shown in the schema diagram.
run_command("DROP TABLE IF EXISTS park;")
ccreate = '''
CREATE TABLE park(
    park_id TEXT PRIMARY KEY,
    name TEXT,
    nickname TEXT,
    city TEXT,
    state TEXT);
    '''
cinsert = '''
INSERT OR IGNORE INTO park
SELECT 
    park_id,
    name,
    aka,
    city,
    state
FROM park_codes;
'''
run_command(ccreate)
run_command(cinsert)

In [16]:
q = "SELECT * FROM park LIMIT 5;"
run_query(q)

,park_id,name,nickname,city,state
0,ALB01,Riverside Park,None,Albany,NY
1,ALT01,Columbia Park,None,Altoona,PA
2,ANA01,Angel Stadium of Anaheim,Edison Field; Anaheim Stadium,Anaheim,CA
3,ARL01,Arlington Stadium,None,Arlington,TX
4,ARL02,Rangers Ballpark in Arlington,The Ballpark in Arlington; Ameriquest Fl,Arlington,TX


In [17]:
#Create the person table with columns and primary key as shown in the schema diagram.
run_command("DROP TABLE IF EXISTS person;")
ccreate_person = '''
    CREATE TABLE person (
        person_id TEXT PRIMARY KEY,
        last_name TEXT,
        first_name TEXT);
        '''
run_command(ccreate_person)
cinsert_person = '''
    INSERT OR IGNORE INTO person
    SELECT 
        id,
        last,
        first
    FROM person_codes;
    '''
run_command(cinsert_person)

In [18]:
q = "SELECT * FROM person LIMIT 5;"
run_query(q)

,person_id,last_name,first_name
0,aardd001,Aardsma,David
1,aaroh101,Aaron,Hank
2,aarot101,Aaron,Tommie
3,aased001,Aase,Don
4,abada001,Abad,Andy


In [19]:
#Create the league table with columns and primary key as shown in the schema diagram.
c1 = '''
    CREATE TABLE IF NOT EXISTS league (
        league_id TEXT PRIMARY KEY,
        name TEXT);
    '''
c2 = '''
    INSERT OR IGNORE INTO league
    Values
    ("NL", "National League"),
    ("AL", "American League"),
    ("AA", "American Association"),
    ("FL", "Federal League"),
    ("PL", "Players League"),
    ("UA", "Union Association");
    '''
run_command(c1)
run_command(c2)

q = "SELECT * FROM league;"
run_query(q)
    

,league_id,name
0,NL,National League
1,AL,American League
2,AA,American Association
3,FL,Federal League
4,PL,Players League
5,UA,Union Association


In [20]:
#Create the appearance_type table with columns and primary key as shown in the schema diagram.
c1 = '''
    CREATE TABLE IF NOT EXISTS appearance_type (
        appearance_type_id TEXT PRIMARY KEY,
        name TEXT,
        category TEXT);
    '''
appearance_type = pd.read_csv("appearance_type.csv")
appearance_type.to_sql("appearance_types", conn, index = False)
c2 = '''
    INSERT OR IGNORE INTO appearance_type
        SELECT 
            appearance_type_id,
            name,
            category
        FROM appearance_types;
    '''
q = "SELECT * FROM appearance_type;"

run_command(c1)
run_command(c2)
run_query(q)
    

,appearance_type_id,name,category
0,O1,Batter 1,offense
1,O2,Batter 2,offense
2,O3,Batter 3,offense
3,O4,Batter 4,offense
4,O5,Batter 5,offense
5,O6,Batter 6,offense
6,O7,Batter 7,offense
7,O8,Batter 8,offense
8,O9,Batter 9,offense
9,D1,Pitcher,defense


## Adding The Team and Game Tables

Now that we have added all of the tables that don't have foreign key relationships, lets add the next two tables. The game and team tables need to exist before our two appearance tables are created. 

Here are some notes on the normalization choices made with each of these tables:

   + team: The start, end, and sequence columns can be derived from the game level data.
   + game: 
      + We have chosen to include all columns for the game log that don't refer to one specific team or player, instead putting those in two appearance tables.
      + We have removed the column with the day of the week, as this can be derived from the date.
      + We have changed the day_night column to day, with the intention of making this a boolean column. Even though SQLite doesn't support the BOOLEAN type, we can use this when creating our table and SQLite will manage the underlying types behind the scenes.
      
Let's create the team and game tables.

In [21]:
def run_command(c):
    with sqlite3.connect("mlb.db") as conn:
        #Enable enforcement of foreign key restraints
        conn.execute('PRAGMA foreign_keys = ON;')
        conn.isolation_level = None
        conn.execute(c)

In [22]:
#Create the team table with columns, primary key, and foreign key as shown in the schema diagram.
c1 = '''
    CREATE TABLE IF NOT EXISTS team (
        team_id TEXT PRIMARY KEY,
        league_id TEXT,
        city TEXT,
        nickname TEXT,
        franch_id TEXT,
        FOREIGN KEY (league_id) REFERENCES league(league_id));
        '''
c2 = '''
    INSERT OR IGNORE INTO team
    SELECT
        team_id,
        league,
        city,
        nickname,
        franch_id
    FROM team_codes;
    '''
q = "SELECT * FROM team LIMIT 5;"

run_command(c1)
run_command(c2)
run_query(q)

,team_id,league_id,city,nickname,franch_id
0,ALT,UA,Altoona,Mountain Cities,ALT
1,ARI,NL,Arizona,Diamondbacks,ARI
2,BFN,NL,Buffalo,Bisons,BFN
3,BFP,PL,Buffalo,Bisons,BFP
4,BL1,None,Baltimore,Canaries,BL1


In [23]:
#Create the game table with columns, primary key, and foreign key as shown in the schema diagram.
c1 = '''
    CREATE TABLE IF NOT EXISTS game (
        game_id TEXT PRIMARY KEY,
        data DATETIME,
        number_of_game INTEGER,
        length_outs INTEGER,
        day BOOLEAN,
        completion TEXT,
        forfeit TEXT,
        protest TEXT,
        park_id TEXT,
        attendance INTEGER,
        length_minutes INTEGER,
        additional_info TEXT,
        acquisition_info TEXT,
        FOREIGN KEY (park_id) REFERENCES park(park_id)
        );
        '''
c2 = '''
    INSERT OR IGNORE INTO game
    SELECT 
        game_id,
        date,
        number_of_game,
        length_outs,
        CASE
            WHEN day_night = "D" THEN 1
            WHEN day_night = "N" THEN 0
            ELSE NULL
            END
            AS day,
        completion,
        forefeit,
        protest,
        park_id,
        attendance,
        length_minutes,
        additional_info,
        acquisition_info
    FROM game_log;
    '''
q = "SELECT * FROM game LIMIT 5;"

run_command(c1)
run_command(c2)
run_query(q)

,game_id,data,number_of_game,length_outs,day,completion,forfeit,protest,park_id,attendance,length_minutes,additional_info,acquisition_info
0,FW1187105040,18710504,0,54,1,None,None,None,FOR01,200,120,None,Y
1,WS3187105050,18710505,0,54,1,None,None,None,WAS01,5000,145,HTBF,Y
2,RC1187105060,18710506,0,54,1,None,None,None,RCK01,1000,140,None,Y
3,CH1187105080,18710508,0,54,1,None,None,None,CHI01,5000,150,None,Y
4,TRO187105090,18710509,0,54,1,None,None,None,TRO01,3250,145,HTBF,Y


## Adding the Team Appearance Table

Our next task is to add the game_team (team appearance) table.

The game_team table has a compound primary key composed of the team name and the game ID. In addition, a boolean column home is used to differentiate between the home and the visiting team. The rest of the columns are scores or statistics that in our original game log are repeated for each of the home and visiting teams.

In order to insert this data cleanly, we'll need to use a UNION clause.

In [24]:
c1 = '''
    CREATE TABLE IF NOT EXISTS game_team (
        game_id TEXT,
        team_id TEXT,
        home BOOLEAN,
        game_number INTEGER,
        league_id TEXT,
        score INTEGER,
        line_score TEXT,
        at_bats INTEGER,
        hits INTEGER,
        doubles INTEGER,
        triples INTEGER,
        homeruns INTEGER,
        rbi INTEGER,
        sacrifice_hits INTEGER,
        sacrifice_flies INTEGER,
        hit_by_pitch INTEGER,
        walks INTEGER,
        intentional_walks INTEGER,
        strikeouts INTEGER,
        stolen_bases INTEGER,
        caught_stealing INTEGER,
        grounded_into_double INTEGER,
        first_catcher_interference INTEGER,
        left_on_base INTEGER,
        pitchers_used INTEGER,
        individual_earned_runs INTEGER,
        team_earned_runs INTEGER,
        wild_pitches INTEGER,
        balks INTEGER,
        putouts INTEGER,
        assists INTEGER,
        errors INTEGER,
        passed_balls INTEGER,
        double_plays INTEGER,
        triple_plays INTEGER,
        PRIMARY KEY (game_id, team_id),
        FOREIGN KEY (game_id) REFERENCES game(game_id),
        FOREIGN KEY (team_id) REFERENCES team(team_id)
    );
    '''
c2 = '''
    INSERT OR IGNORE INTO game_team
    SELECT
        game_id,
        h_name,
        1 AS home,
        h_game_number,
        h_league,
        h_score,
        h_line_score,
        h_at_bats,
        h_hits,
        h_doubles,
        h_triples,
        h_homeruns,
        h_rbi,
        h_sacrifice_hits,
        h_sacrifice_flies,
        h_hit_by_pitch,
        h_walks,
        h_intentional_walks,
        h_strikeouts,
        h_stolen_bases,
        h_caught_stealing,
        h_grounded_into_double,
        h_first_catcher_interference,
        h_left_on_base,
        h_pitchers_used,
        h_individual_earned_runs,
        h_team_earned_runs,
        h_wild_pitches,
        h_balks,
        h_putouts,
        h_assists,
        h_errors,
        h_passed_balls,
        h_double_plays,
        h_triple_plays
    FROM game_log
    
UNION

    SELECT
        game_id,
        v_name,
        0 AS home,
        v_game_number,
        v_league,
        v_score,
        v_line_score,
        v_at_bats,
        v_hits,
        v_doubles,
        v_triples,
        v_homeruns,
        v_rbi,
        v_sacrifice_hits,
        v_sacrifice_flies,
        v_hit_by_pitch,
        v_walks,
        v_intentional_walks,
        v_strikeouts,
        v_stolen_bases,
        v_caught_stealing,
        v_grounded_into_double,
        v_first_catcher_interference,
        v_left_on_base,
        v_pitchers_used,
        v_individual_earned_runs,
        v_team_earned_runs,
        v_wild_pitches,
        v_balks,
        v_putouts,
        v_assists,
        v_errors,
        v_passed_balls,
        v_double_plays,
        v_triple_plays
    FROM game_log;
    '''

run_command(c1)
run_command(c2)
        

In [25]:
q = '''
    SELECT *
    FROM game_team
    WHERE team_id = "CL1" AND score = 0;
    '''
run_query(q)

,game_id,team_id,home,game_number,league_id,score,line_score,at_bats,hits,doubles,triples,homeruns,rbi,sacrifice_hits,sacrifice_flies,hit_by_pitch,walks,intentional_walks,strikeouts,stolen_bases,caught_stealing,grounded_into_double,first_catcher_interference,left_on_base,pitchers_used,individual_earned_runs,team_earned_runs,wild_pitches,balks,putouts,assists,errors,passed_balls,double_plays,triple_plays
0,BS1187207010,CL1,0,14,None,0,000000000,36,8,1,0,0,0,0,0,0,0,None,0,0,0.0,0,None,9,1,11,0,1,0,27,12,NaN,2,0,0
1,FW1187105040,CL1,0,1,None,0,000000000,30,4,1,0,0,0,0,0,0,1,None,6,1,NaN,-1,None,4,1,1,1,0,0,27,9,0.0,3,0,0


## Adding the Person Appearance Table

The final table we need to create is person_appearance. The person_appearance table will be used to store information on appearances in games by managers, players, and umpires as detailed in the appearance_type table.

We'll need to use a similar technique to insert data as we used with the team_appearance table, however we will have to write much larger queries - one for each column instead of one for each team as before. We will need to work out for each column what the appearance_type_id will be by cross-referencing the columns with the appearance_type table.

We have decided to create an integer primary key for this table, because having every column be a compound primary quickly becomes cumbersome when writing queries.

In [26]:
#Create the person_appearance table with columns, primary key, and foreign keys as shown in the schema diagram.
c0 = "DROP TABLE IF EXISTS person_appearance"

run_command(c0)
c1 = '''
    CREATE TABLE IF NOT EXISTS person_appearance (
        appearance_id INTEGER PRIMARY KEY,
        person_id TEXT,
        team_id TEXT,
        game_id TEXT,
        appearance_type_id TEXT,
        FOREIGN KEY (person_id) REFERENCES person(person_id),
        FOREIGN KEY (team_id) REFERENCES team(team_id),
        FOREIGN KEY (game_id) REFERENCES game(game_id),
        FOREIGN KEY (appearance_type_id) REFERENCES appearance_type(appearance_type_id)    
    );
    '''
c2 = '''
    INSERT OR IGNORE INTO person_appearance (
        game_id,
        team_id,
        person_id,
        appearance_type_id
    )
    SELECT 
        game_id,
        NULL,
        hp_umpire_id,
        "UHP"
    FROM game_log
    WHERE hp_umpire_id IS NOT NULL
    
UNION
    
    SELECT
        game_id,
        NULL,
        [1b_umpire_id],
        "U1B"
    FROM game_log
    WHERE [1b_umpire_id] IS NOT NULL
    
UNION
 
    SELECT
        game_id,
        NULL,
        [2b_umpire_id],
        "U2B"
    FROM game_log
    WHERE [2b_umpire_id] IS NOT NULL
    
UNION

    SELECT
        game_id,
        NULL,
        [3b_umpire_id],
        "U3B"
    FROM game_log
    WHERE [3b_umpire_id] IS NOT NULL
    
UNION

    SELECT 
        game_id,
        NULL,
        lf_umpire_id,
        "ULF"
    FROM game_log
    WHERE lf_umpire_id IS NOT NULL
    
UNION

    SELECT 
        game_id,
        NULL,
        rf_umpire_id,
        "URF"
    FROM game_log
    WHERE rf_umpire_id IS NOT NULL
    
UNION

    SELECT
        game_id,
        v_name,
        v_manager_id,
        "MM"
    FROM game_log
    WHERE v_manager_id IS NOT NULL
    
UNION

    SELECT 
        game_id,
        h_name,
        h_manager_id,
        "MM"
    FROM game_log
    WHERE h_manager_id IS NOT NULL
    
UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        winning_pitcher_id,
        "AWP"
    FROM game_log
    WHERE winning_pitcher_id IS NOT NULL
    
UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN v_name
            ELSE h_name
            END,
        losing_pitcher_id,
        "ALP"
    FROM game_log
    WHERE losing_pitcher_id IS NOT NULL
    
UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        saving_pitcher_id,
        "ASP"
    FROM game_log
    WHERE saving_pitcher_id IS NOT NULL
    
UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        winning_rbi_batter_id,
        "AWB"
    FROM game_log
    WHERE winning_rbi_batter_id IS NOT NULL
    
UNION

    SELECT
        game_id,
        h_name,
        h_starting_pitcher_id,
        "PSP"
    FROM game_log
    WHERE h_starting_pitcher_id IS NOT NULL
    
UNION

    SELECT 
        game_id,
        v_name,
        v_starting_pitcher_id,
        "PSP"
    FROM game_log
    WHERE v_starting_pitcher_id IS NOT NULL;
    '''

template = '''
INSERT INTO person_appearance (
    game_id,
    team_id,
    person_id,
    appearance_type_id
) 
    SELECT
        game_id,
        {hv}_name,
        {hv}_player_{num}_id,
        "O{num}"
    FROM game_log
    WHERE {hv}_player_{num}_id IS NOT NULL

UNION

    SELECT
        game_id,
        {hv}_name,
        {hv}_player_{num}_id,
        "D" || CAST({hv}_player_{num}_def_pos AS INT)
    FROM game_log
    WHERE {hv}_player_{num}_id IS NOT NULL;
'''

run_command(c1)
run_command(c2)

for hv in ["h","v"]:
    for num in range(1,10):
        query_vars = {
            "hv": hv,
            "num": num
        }
        run_command(template.format(**query_vars))

In [27]:
print(run_query("SELECT COUNT(DISTINCT game_id) games_game FROM game;"))
print(run_query("SELECT COUNT(DISTINCT game_id) person_appearance_game FROM person_appearance;"))

   games_game
0      171907
   person_appearance_game
0                  171907


In [28]:
q = '''
    SELECT 
        pa.*,
        at.name,
        at.category
    FROM person_appearance pa
    INNER JOIN appearance_type at ON at.appearance_type_id = pa.appearance_type_id
    WHERE pa.game_id = "WAS201610020"
    ORDER BY team_id, appearance_type_id;
    '''
run_query(q)

,appearance_id,person_id,team_id,game_id,appearance_type_id,name,category
0,1591102,porta901,None,WAS201610020,U1B,First Base,umpire
1,1591101,onorb901,None,WAS201610020,U2B,Second Base,umpire
2,1591100,kellj901,None,WAS201610020,U3B,Third Base,umpire
3,1591103,tumpj901,None,WAS201610020,UHP,Home Plate,umpire
4,1591104,brica001,MIA,WAS201610020,ALP,Losing Pitcher,award
5,6706897,koeht001,MIA,WAS201610020,D1,Pitcher,defense
6,4735171,telit001,MIA,WAS201610020,D2,Catcher,defense
7,5580199,bourj002,MIA,WAS201610020,D3,1st Base,defense
8,4453495,gordd002,MIA,WAS201610020,D4,2nd Base,defense
9,5016847,pradm001,MIA,WAS201610020,D5,3rd Base,defense


## Removing the Original Tables

We've now created all normalized tables and inserted all of our data!

Our last task is to remove the tables we created to import the original CSVs.

In [29]:
show_tables()

,name,type
0,league,table
1,appearance_type,table
2,team,table
3,game,table
4,game_team,table
5,game_log,table
6,person_codes,table
7,park_codes,table
8,team_codes,table
9,park,table


In [30]:
tables = ["appearance_types", 
          "person_codes",
          "team_codes",
          "park_codes",
          "game_log"]
for table in tables:
    run_command("DROP TABLE {}".format(table))
    
show_tables()

,name,type
0,league,table
1,appearance_type,table
2,team,table
3,game,table
4,game_team,table
5,park,table
6,person,table
7,person_appearance,table
